In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm

In [2]:
df=pd.read_csv('Base.csv')

In [3]:
df=df.sort_values(by=['Date'])

In [5]:
df

,Date,SPX Index,VIX Index (L1),Freddie Mac 30 Year Mortgage Rate,U.S. 5-year Treasury constant maturity rate,CDX HY Spread,CDX IG Spread
2860,2012-08-09,1402.80,15.28,3.64,0.74,551.268,103.000
2859,2012-08-10,1405.87,14.74,3.61,0.71,552.893,102.751
2858,2012-08-13,1404.11,13.70,3.56,0.71,554.523,102.750
2857,2012-08-14,1403.93,14.85,3.57,0.75,553.750,103.000
2856,2012-08-15,1405.53,14.63,3.65,0.80,558.182,103.152
...,...,...,...,...,...,...,...
4,2024-02-12,5021.84,13.93,7.14,4.13,345.338,53.619
3,2024-02-13,4953.17,15.85,7.25,4.31,354.456,55.056
2,2024-02-14,5000.62,14.38,7.29,4.25,350.427,54.015
1,2024-02-15,5029.73,14.01,7.24,4.22,341.624,52.343


In [6]:
if 'Date' in df.columns:
    keep_date = True
    dates = df['Date']  # Preserve date column
else:
    keep_date = False

# Select only numeric columns for HP filter application
numeric_columns = df.select_dtypes(include=['number']).columns

# Apply the HP filter to each numeric column and store the results
lambda_hp = 14400  # Adjust this value as needed based on your data's frequency
#changed lambda to monthly -> 1600 to 14440; 3/22/2024
hp_results = pd.DataFrame()
cyclical_components=pd.DataFrame()

for column in numeric_columns:
    cycle, trend = sm.tsa.filters.hpfilter(df[column], lamb=lambda_hp)
    cyclical_components[column + '_cycle'] = cycle
    hp_results[f'{column}_trend'] = trend
    hp_results[f'{column}_cycle'] = cycle

# If you preserved dates, add them back to the results DataFrame
if keep_date:
    hp_results['Date'] = dates

# Reorder columns to put 'date' first if it was preserved
if 'Date' in hp_results.columns:
    cols = ['Date'] + [col for col in hp_results.columns if col != 'Date']
    hp_results = hp_results[cols]

# Save the results to a new CSV file
hp_results.to_csv('hp_filtered_data.csv', index=False)  # Use index=False to exclude DataFrame index if 'date' is a column
cyclical_components.to_csv('cyclical_components.csv', index=False)

print('HP filter applied to all numeric features. Results, including dates, saved to hp_filtered_data.csv')

HP filter applied to all numeric features. Results, including dates, saved to hp_filtered_data.csv
